In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import looptools.pll as pll
import looptools.auxiliary as aux
from looptools.nprolaserlock import NPROLaserLock
from looptools.simulation import loop_crossover_optimizer

In [ ]:
sps = 80e6 # PLL clock rate
frfr = np.logspace(np.log10(1e-6), np.log10(1e6), int(1e5)) # Fourier frequency array (Hz)

In [ ]:
parameters_pll = {
    "Amp": 1e-5,         # normalized signal amplitude
    "Cshift": 19,        # Gain bit shift (right)
    "Klf": 5,            # LF bit shift (right) 5
    "twostages":True,    # Two stages for LF
    "Kp": 18,            # P bit shift (left) 18
    "Ki": 4,             # I bit shift (left) 4
    "n_reg": 10          # Number of registers
}

pll_1 = pll.PLL(
    sps, 
    parameters_pll["Amp"], 
    parameters_pll["Cshift"], 
    parameters_pll["Klf"], 
    parameters_pll["Kp"], 
    parameters_pll["Ki"], 
    twostages=parameters_pll["twostages"],
    n_reg=parameters_pll["n_reg"]
)

ugf, margin = aux.get_margin(pll_1.Gf(f=frfr), frfr, deg=True) # compute UGF and phase margin

print(f"Unity gain frequency = {ugf:.4e} Hz; Phase margin = {margin:.4f} degrees")

In [ ]:
ll_parameters = {
    "pll": pll_1,  # PLL instance for the laser lock
    "C1": 64,      # shifted bit at the gain stage 1
    "C2": 64,      # shifted bit at the gain stage 2
    "Ki1": 56,     # I gain of a digital servo for a PZT loop
    "Kii1": 30,    # II gain of a digital servo for a PZT loop
    "Kp2": 0,      # P gain of a digital servo for a temperature loop
    "Ki2": 31,     # I gain of a digital servo for a temperature loop
    "Kdac": 1,     # DAC gain
    "Kc_pzt": 2.5, # gain of an analog filter in a PZT loop (1e3/75 for LISA)
    "Fc_pzt": 1/(2*np.pi*1e3*10e-12), # corner frequency of an analog filter in a PZT loop (1/(2*np.pi*1e3*10e-12) for LISA)
    "Kc_temp": 1,  # gain of an analog filter in a temperature loop
    "Fc_temp": 10, # corner frequency of an analog filter in a temperature loop
    "Ka_pzt": 14e6,       # Laser PZT actuator efficiency [Hz/V]
    "Fa_pzt": 100e5,      # Laser PZT response bandwidth [Hz]
    "Ka_temp": 500e6,     # Laser temperature actuator efficiency
    "Fa_temp": 10,        # Laser temperature response bandwidth [Hz]
    "Nreg1": 0,           # the number of registers which represents, e.g., the delay between IPU and PCU
    "mode": 'frequency',  # Laser lock mode "phase" or "frequency"
    "extrapolate": False, # extrapolate PLL TF or not
}

p_llextra = { # extrapolation parameters
    'Fpll': [True, 1e2,1],
    'p_II1': [True, 1e4,-2],
    't_II1': [True, 1e4,-2],
}

# # : instantiate LaserLock class
ll = NPROLaserLock(
    sps, 
    pll=ll_parameters["pll"], 
    C1=ll_parameters["C1"], 
    C2=ll_parameters["C2"],
    Ki1=ll_parameters["Ki1"],   
    Kii1=ll_parameters["Kii1"],
    Kp2=ll_parameters["Kp2"], 
    Ki2=ll_parameters["Ki2"],
    Kdac=ll_parameters["Kdac"],
    Kc_pzt=ll_parameters["Kc_pzt"], 
    Fc_pzt=ll_parameters["Fc_pzt"],
    Ka_pzt=ll_parameters["Ka_pzt"], 
    Fa_pzt=ll_parameters["Fa_pzt"],
    Kc_temp=ll_parameters["Kc_temp"], 
    Fc_temp=ll_parameters["Fc_temp"],
    Ka_temp=ll_parameters["Ka_temp"], 
    Fa_temp=ll_parameters["Fa_temp"],
    Nreg1=ll_parameters["Nreg1"],
    mode=ll_parameters["mode"],
    extrapolate=p_llextra
)

In [ ]:
ll.pzt.block_diagram(transfer_functions=False)

In [ ]:
fig, ax = ll.pzt.nyquist_plot(np.logspace(0,6,int(1e5)), logx=True, logy=True, arrow_frequency=1000, label='PZT')
ll.temp.nyquist_plot(np.logspace(0,6,int(1e5)), ax=ax, ls='--', critical_point=True, label='Temp')
plt.show()

In [ ]:
fig, axes = ll.pzt.bode_plot(frfr, label='PZT')
ll.temp.bode_plot(frfr, axes=axes, label='Temp', ls='--')
plt.show()

In [ ]:
f_cross = aux.loop_crossover(ll.pzt, ll.temp, frfr)

print(f"Cross-over frequency between the fast and slow loops: {f_cross:.4e} Hz")

In [ ]:
figsize=(6,4)
dpi=300
fontsize=8
linewidth=1.5

fig, (ax1, ax2) = plt.subplots(2,1, figsize=figsize, dpi=dpi)
ax1.loglog(frfr, np.abs(ll.pzt.Gf(f=frfr)), linewidth=linewidth, color="blue", linestyle='-', label="PZT")
ax1.loglog(frfr, np.abs(ll.temp.Gf(f=frfr)), linewidth=linewidth, color="red", linestyle='-', label="Temp")
ax1.loglog(frfr, np.abs(ll.Gf(f=frfr)), linewidth=linewidth, color="black", linestyle='--', label="Total")
ax2.semilogx(frfr, np.angle(ll.pzt.Gf(f=frfr)), linewidth=linewidth, color="blue", linestyle='-', label="PZT")
ax2.semilogx(frfr, np.angle(ll.temp.Gf(f=frfr)), linewidth=linewidth, color="red", linestyle='-', label="Temp")
ax2.semilogx(frfr, np.angle(ll.Gf(f=frfr)), linewidth=linewidth, color="black", linestyle='--', label="Total")
for ax in (ax1, ax2):
    ax.axvline(f_cross, color='orange', ls='-', label='Temp-PZT crossover')
ax2.set_xlabel("Frequency (Hz)", fontsize=fontsize)
ax1.set_ylabel("Magnitude", fontsize=fontsize)
ax2.set_ylabel("Phase (deg)", fontsize=fontsize)
ax1.set_title("Test", fontsize=fontsize)
ax1.axes.xaxis.set_ticklabels([])
ax1.tick_params(labelsize=fontsize)
ax2.tick_params(labelsize=fontsize)
ax1.grid()
ax2.grid()
ax1.set_xlim(frfr[0],frfr[-1])
ax2.set_xlim(frfr[0],frfr[-1])
ax1.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize-1, handlelength=2.9)
fig.tight_layout()
fig.align_ylabels()

In [ ]:
ll.temp.property_list, ll.pzt.property_list

In [ ]:
meta = {"Fctrl2_Ki": [1,50,30,True]}

loop_crossover_optimizer(ll.temp, ll.pzt, frfr, 0.1, meta)

In [ ]:
f_cross = aux.loop_crossover(ll.pzt, ll.temp, frfr)
print(f"Original crossover frequency: {f_cross:.4f} Hz")
ll.temp.Fctrl2_Ki = 33
f_cross = aux.loop_crossover(ll.pzt, ll.temp, frfr)
print(f"New crossover frequency: {f_cross:.4f} Hz")

In [ ]:

figsize=(6,4)
dpi=300
fontsize=8
linewidth=1.5

fig, (ax1, ax2) = plt.subplots(2,1, figsize=figsize, dpi=dpi)
ax1.loglog(frfr, np.abs(ll.pzt.Gf(f=frfr)), linewidth=linewidth, color="blue", linestyle='-', label="PZT")
ax1.loglog(frfr, np.abs(ll.temp.Gf(f=frfr)), linewidth=linewidth, color="red", linestyle='-', label="Temp")
ax1.loglog(frfr, np.abs(ll.Gf(f=frfr)), linewidth=linewidth, color="black", linestyle='--', label="Total")
ax2.semilogx(frfr, np.angle(ll.pzt.Gf(f=frfr)), linewidth=linewidth, color="blue", linestyle='-', label="PZT")
ax2.semilogx(frfr, np.angle(ll.temp.Gf(f=frfr)), linewidth=linewidth, color="red", linestyle='-', label="Temp")
ax2.semilogx(frfr, np.angle(ll.Gf(f=frfr)), linewidth=linewidth, color="black", linestyle='--', label="Total")
for ax in (ax1, ax2):
    ax.axvline(f_cross, color='orange', ls='-', label='Temp-PZT crossover')
ax2.set_xlabel("Frequency (Hz)", fontsize=fontsize)
ax1.set_ylabel("Magnitude", fontsize=fontsize)
ax2.set_ylabel("Phase (deg)", fontsize=fontsize)
ax1.set_title("Test", fontsize=fontsize)
ax1.axes.xaxis.set_ticklabels([])
ax1.tick_params(labelsize=fontsize)
ax2.tick_params(labelsize=fontsize)
ax1.grid()
ax2.grid()
ax1.set_xlim(frfr[0],frfr[-1])
ax2.set_xlim(frfr[0],frfr[-1])
ax1.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize-1, handlelength=2.9)
fig.tight_layout()
fig.align_ylabels()
plt.show()